# **LangChain Memory**

Most LLM applications have a conversational interface. An essential component of a conversation is being able to refer to information introduced earlier in the conversation. At bare minimum, a conversational system should be able to access some window of past messages directly. A more complex system will need to have a world model that it is constantly updating, which allows it to do things like maintain information about entities and their relationships.

We call this ability to store information about past interactions "memory". LangChain provides a lot of utilities for adding memory to a system. These utilities can be used by themselves or incorporated seamlessly into a chain.

## **Building memory into a system**
The two core design decisions in any memory system are:
- How state is stored
- How state is queried

## **What's covered?**
- ConversationBufferMemory
- End-to-end Example: Conversational AI Bot
- Saving and Loading a Chat History
- ConversationBufferWindowMemory

## **ConversationBufferMemory**

Let's take a look at how to use ConversationBufferMemory in chains. ConversationBufferMemory is an extremely simple form of memory that just keeps a list of chat messages in a buffer and passes those into the prompt template.

This memory type can be connected to a conversation. It allows for storing messages and then extracts the messages in a variable.

## **End-to-end Example: Conversational AI Bot**

<img src="images/memory.png">

### **Steps:**
1. Import Chat Model and Configure the API Key
2. Create Chat Template
3. Initialize the Memory
4. Create a Output Parser
5. Build a Chain
6. Invoke the chain with human_input and chat_history
7. Saving to memory
8. Run Step 6 and 7 in a loop

# Step 1 : Import Chat Model and Configure the API Key

In [1]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 993.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [2]:
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 23.0 MB/s eta 0:00:00


# Step 1 - Import Chat Model and Configure the API Key

In [6]:
from langchain_openai import ChatOpenAI

from google.colab import userdata  # api_key has uploaded in google colab
API_Key =userdata.get('OpenAIAPIKey')
chat_model = ChatOpenAI(openai_api_key =API_Key)

# Step 2  -Create Chat Template

In [7]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder,SystemMessagePromptTemplate,HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

### ************* Example Starts****************

In [8]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()
memory.load_memory_variables({})

<ipython-input-8-ac7c6ea998a8>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


{'history': ''}

In [9]:
memory.buffer

''

In [10]:
memory = ConversationBufferMemory(memory_key = 'chat_history')
memory.chat_memory.add_user_message("Hi")
memory.chat_memory.add_ai_message("Hi, What's up? How can i help you")
memory.load_memory_variables({})

{'chat_history': "Human: Hi\nAI: Hi, What's up? How can i help you"}

In [11]:
memory.buffer

"Human: Hi\nAI: Hi, What's up? How can i help you"

In [12]:
memory = ConversationBufferMemory(memory_key = 'chat_history',return_messages = True)
memory.chat_memory.add_user_message("Hi")
memory.chat_memory.add_ai_message("Hi, What's up? How can i help you")
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hi, What's up? How can i help you", additional_kwargs={}, response_metadata={})]}

### ***************** Example Ends *******************

In [13]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    SystemMessage(content = "you are a chatbot having a conversation with a human"),
    # Creating a chat_history placeholder
    MessagesPlaceholder(variable_name = 'chat_history'),
    #Human Prompt
    HumanMessagePromptTemplate.from_template("{human_input}"),])


# Step 3- Initialize the Memory

In [14]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key = 'chat_history',return_messages = True)


# Step 4 - Create a Output Parser

In [15]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

# Step 5 -Build a Chain(New Method)

In [16]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# Define a function to load the message from memory
def get_messages_from_memory(human_input):
    return memory.load_memory_variables(human_input)['chat_history']

# Define a chain
chain = RunnablePassthrough.assign(chat_history = RunnableLambda(get_messages_from_memory)) |  chat_prompt_template | chat_model | output_parser

# Step 6 -Invoke the chain with human_input and chat_history

In [17]:
query= {"human_input":"Hi, How are you?"}
response = chain.invoke(query)
response

"Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to chat with you. How are you doing today?"

# Step 7 -Saving to Memory

In [18]:
memory.save_context(query, {'output':response})
memory.buffer

[HumanMessage(content='Hi, How are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to chat with you. How are you doing today?", additional_kwargs={}, response_metadata={})]

# Step 8 -Run Step 6 and 7 in a loop

In [19]:
while True:
  query = {"human_input":input("Enter a query:  ")}
  print(f"User:{query['human_input']}")
  if query['human_input'] in ['bye','quit','exit']:
    break
  response = chain.invoke(query)
  print(f"AI:{response}")
  memory.save_context(query,{'output':response})

Enter a query:  hi,how are you
User:hi,how are you
AI:I'm just a bot, so I don't have feelings, but I'm here to help you with any questions or just have a chat. How can I assist you today?
Enter a query:  my name is achyuth, i want you to help with data wraningling  techniques
User:my name is achyuth, i want you to help with data wraningling  techniques
AI:Of course, Achyuth! I'd be happy to help you with data wrangling techniques. Data wrangling involves cleaning, transforming, and organizing raw data into a usable format for analysis. 

Some common data wrangling techniques include:
- Handling missing values
- Removing duplicates
- Standardizing data formats
- Merging datasets
- Filtering and sorting data
- Creating new variables
- Handling outliers

Is there a specific aspect of data wrangling you'd like to learn more about or need help with? Feel free to ask any questions you have!
Enter a query:  explain oops concept in 5lines and why we need to learn that
User:explain oops concep

In [20]:
# check memory(chat history)

memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='Hi, How are you?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to chat with you. How are you doing today?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='hi,how are you', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I'm just a bot, so I don't have feelings, but I'm here to help you with any questions or just have a chat. How can I assist you today?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='my name is achyuth, i want you to help with data wraningling  techniques', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Of course, Achyuth! I'd be happy to help you with data wrangling techniques. Data wrangling involves cleaning, transforming, and organizing raw data into a usable format for analysis. \n\nSome common data wrangling techniques include:\n- Handling

In [21]:
memory.buffer

[HumanMessage(content='Hi, How are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to chat with you. How are you doing today?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi,how are you', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm just a bot, so I don't have feelings, but I'm here to help you with any questions or just have a chat. How can I assist you today?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='my name is achyuth, i want you to help with data wraningling  techniques', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Of course, Achyuth! I'd be happy to help you with data wrangling techniques. Data wrangling involves cleaning, transforming, and organizing raw data into a usable format for analysis. \n\nSome common data wrangling techniques include:\n- Handling missing values\n- Rem

# Saving a Chat History

In [22]:
import pickle

In [23]:
chat_history = pickle.dumps(memory)
with open("conversation_memory.pkl",'wb') as f:
  f.write(chat_history)

# Loading a Chat History


In [25]:
chat_history_loaded = pickle.load(open("/content/conversation_memory.pkl",'rb'))
print(chat_history_loaded)

chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, How are you?', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to chat with you. How are you doing today?", additional_kwargs={}, response_metadata={}), HumanMessage(content='hi,how are you', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm just a bot, so I don't have feelings, but I'm here to help you with any questions or just have a chat. How can I assist you today?", additional_kwargs={}, response_metadata={}), HumanMessage(content='my name is achyuth, i want you to help with data wraningling  techniques', additional_kwargs={}, response_metadata={}), AIMessage(content="Of course, Achyuth! I'd be happy to help you with data wrangling techniques. Data wrangling involves cleaning, transforming, and organizing raw data into a usable format for analysis. \n\nSome common data wrangling techniques

## **SQLChatMessageHistory**

`ChatMessageHistory` allows us to store separate conversation histories per user or session which is often done by the real-time chatbots. `session_id` is used to distinguish between separate conversations.

In order to use it, we can use a `get_session_history` function which take `session_id` and returns a message history object.

There is a support of many `Memory` components under `langchain_community.chat_message_histories`, like:
1. AstraDBChatMessageHistory
2. DynamoDBChatMessageHistory
3. CassandraChatMessageHistory
4. ElasticsearchChatMessageHistory
5. KafkaChatMessageHistory
6. MongoDBChatMessageHistory
7. RedisChatMessageHistory
8. PostgresChatMessageHistory
9. SQLChatMessageHistory

**[Click Here](https://python.langchain.com/v0.2/docs/integrations/memory/)** to read more.

### **Usage**

To use the storage you need to provide only 2 things:

1. Session Id - a unique identifier of the session, like user name, email, chat id etc.
2. Connection string
    - For SQL (SQLAlchemy) - A string that specifies the database connection. It will be passed to SQLAlchemy create_engine function.
    - For SQLite - A string that specifies the database connection. For SQLite, that string is slqlite:/// followed by the name of the database file. If that file doesn't exist, it will be created.

In [26]:
from langchain_openai import ChatOpenAI

from google.colab import userdata  # api_key has uploaded in google colab
API_Key =userdata.get('OpenAIAPIKey')
chat_model = ChatOpenAI(openai_api_key =API_Key)

In [27]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.5 MB/s eta 0:00:00


In [46]:
import os

# Create directory if it doesn't exist
os.makedirs("/content/chats_data", exist_ok=True)

In [48]:
#from langchain_community.chat_message_histories import SQLChatMessageHistory

#def get_session_message_history_from_db(session_id):
#  chat_message_history = SQLChatMessageHistory(session_id = session_id,connection = 'sqlite:///chats_data/sqlite.db')
#  return chat_message_history
# the above code for jupyter notebook


from langchain_community.chat_message_histories import SQLChatMessageHistory

# Function to get chat message history from the SQLite database
def get_session_message_history_from_db(session_id):
    db_path = "/content/chats_data/sqlite.db"  # Specify the correct path
    connection_string = f"sqlite:///{db_path}"

    # Initialize SQLChatMessageHistory with the session ID and connection string
    chat_message_history = SQLChatMessageHistory(session_id=session_id, connection=connection_string)
    return chat_message_history

In [49]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

chat_template = ChatPromptTemplate.from_messages([('system',"you are a helpful AI assistant"),
                                                  MessagesPlaceholder(variable_name = 'history'),
                                                  ("human","{human_input}")])

In [50]:
# chain

chain = chat_template | chat_model

In [51]:
from langchain_core.runnables.history import RunnableWithMessageHistory

# Define a chain

conversation_chain = RunnableWithMessageHistory(chain,get_session_message_history_from_db,
                                                input_messages_key = 'human_input',
                                                history_messages_key = "history")

In [52]:
# This is where we configure the session id

user_id = "Achyuth"
config = {"configurable":{"session_id":user_id}}
input_prompt = {"human_input":"My name is Achyuth, can you tell me prime minister of India?"}
response = conversation_chain.invoke(input_prompt,config = config)
response.content

'The current Prime Minister of India is Narendra Modi.'

In [53]:
user_id = "Ak"
config = {"configurable":{"session_id":user_id}}
input_prompt = {"human_input":"My name is Ak, who is MS dhoni"}
response = conversation_chain.invoke(input_prompt,config = config)
response.content

'Mahendra Singh Dhoni, commonly known as MS Dhoni, is a former Indian international cricketer and one of the most successful captains in Indian cricket history. He is considered one of the greatest wicketkeeper-batsmen in the world and is known for his calm demeanor on the field, sharp cricketing acumen, and finishing abilities. Dhoni led the Indian cricket team to several victories, including the 2007 ICC World Twenty20, the 2010 and 2016 Asia Cups, the 2011 ICC Cricket World Cup, and the 2013 ICC Champions Trophy. He retired from international cricket in 2020.'

In [54]:
def chat_bot(session_id,prompt):
  config = {"configurable":{"session_id":user_id}}
  input_prompt = {"human_input":prompt}
  response = conversation_chain.invoke(input_prompt, config = config)
  return response.content

In [55]:
user_id = "Achyuth"
input_prompt = "Do you remember my name?"
chat_bot(session_id = user_id,prompt = input_prompt)

'Yes, your name is Achyuth. How can I assist you further, Achyuth?'

In [56]:
user_id = "kumar"
input_prompt = "Do you remember my name?"
chat_bot(session_id = user_id,prompt = input_prompt)

"I'm sorry, but I don't have the ability to remember specific user information. How can I assist you today?"

In [59]:
user_id = "ak"
input_prompt = "Do you remember my name?"
chat_bot(session_id = user_id,prompt = input_prompt)

"I'm sorry, I don't have the ability to remember personal information like names. How can I assist you today?"

In [60]:
# This is where we configure the session id

user_id = "Achyuth"
config = {"configurable":{"session_id":user_id}}
input_prompt = {"human_input":"My name is Achyuth, which state more scope for data science jobs in india?"}
response = conversation_chain.invoke(input_prompt,config = config)
response.content

'In India, some of the states that have a strong presence of data science jobs and opportunities are Karnataka (particularly in cities like Bangalore), Maharashtra (especially in Mumbai and Pune), Telangana (with Hyderabad being a hub for tech companies), and Delhi NCR region. These states have a thriving IT industry and a growing demand for data science professionals.'

In [61]:
# This is where we configure the session id

user_id = "Achyuth"
config = {"configurable":{"session_id":user_id}}
input_prompt = {"human_input":"My name is Achyuth, which state more scope for data science jobs in india?"}
response = conversation_chain.invoke(input_prompt,config = config)
print(response.content)

In India, states like Karnataka (especially Bangalore), Maharashtra (Mumbai and Pune), Telangana (Hyderabad), Delhi NCR region, and Tamil Nadu (Chennai) have a significant scope for data science jobs. These states have a strong presence of IT companies, startups, and industries that are actively hiring data science professionals.
